In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt

In [2]:
@jit
def dprod(A,B):
    nobsa,na= A.shape
    nobsb,nb= B.shape
    Res= np.empty((nobsa,nb*na))
    if (nobsa!=nobsb):
        return 'A and B must have same number of rows'
    for ia in range(na):
        for ib in range(nb):
            for t in range(nobsa):
                Res[t,nb*(ia-1)+ib]=A[t,ia] * B[t, ib]
    return Res

In [3]:
@jit
def Chebnodes(p, nodetype=0):
    n, a, b = p[0], p[1], p[2]
    s = (b-a) / 2 
    m = (b+a) / 2  
    if (nodetype < 2):  # usual nodes
        k = np.pi*np.linspace(0.5,n-0.5,n)  
        x = m - np.cos(k[0:n]/n) * s  
        if (nodetype == 1):  # Extend nodes to endpoints
            aa = x[0]  
            bb = x[-1]  
            x = (bb*a - aa*b)/(bb-aa) + (b-a)/(bb-aa)*x
    else: # Lobatto nodes
        k = np.pi*np.linspace(0,n-1,n)
        x = m - np.cos(k[0:n]/(n-1)) * s
    return x

In [4]:
@jit
def evalbasex(p, x):
    n, a, b = p[0], p[1], p[2]
    z = (2/(b-a)) * (x-(a+b)/2)
    m = z.shape[0]
    bas = np.empty((m, n));
    bas[:, 0] = 1.0
    bas[:, 1] = z[:]
    z = 2 * z
    for j in range(2,n):
        for i in range(m):
            bas[i, j] = z[i] * bas[i, j-1] - bas[i, j-2]
    return bas

In [ ]:
def derivative_op(p, order=1):
    n, a, b = p[0], p[1], p[2]
    if (order > 0):
        D = np.empty(max(2, order), 1)  
        i = repeat(collect(1:n), outer=[1, n]); j = i'  # 50

        # 51
        inds = find((rem(i + j, 2) .== 1) & (j.>i))
        r, c = similar(inds), similar(inds)
        for i in 1:length(inds)
            r[i], c[i] = ind2sub((n, n), inds[i])
        end

        d = sparse(r, c, (4/(b-a)) * (vec(j[1, c])-1), n-1, n)  # 52
        d[1, :] ./= 2  # 53
        D[1] = d  # 54
        for ii=2:max(2, order)
            D[ii] = d[1:n-ii, 1:n-ii+1] * D[ii-1]  # 56
        end
    else
        D = cell(abs(order), 1)  # 64
        nn = n - order  # 65
        i = (0.25 * (b - a)) ./(1:nn)  # 66
        d = sparse(vcat(1:nn, 1:nn-2), vcat(1:nn, 3:nn), vcat(i, -i[1:nn-2]),
                   nn, nn)  # 67
        d[1, 1] *= 2  # 68
        d0 = ((-1).^(0:nn-1)') .* sum(d, 1)  # 69
        D[1] = sparse(vcat(d0[1:n]', d[1:n, 1:n]))  # 70
        for ii=-2:-1:order
            ind = 1:n-ii-1
            D[-ii] = sparse(vcat(d0[ind]', d[ind, ind]) * D[-ii-1])
        end
    end
    D, n-order, a, b, Any[n, a, b]
end

In [6]:
A1=np.random.randn(10,10)
B1=np.random.randn(10,100000)
dprod(A1,B1)

array([[ 0.04262418,  0.58903612, -0.02798125, ..., -0.12700451,
         0.49984957,  0.63626028],
       [-0.09900679,  0.00371128,  0.03417576, ...,  0.92144505,
        -1.62579291, -0.17715555],
       [ 0.08958156, -0.10634781, -0.00622043, ...,  0.0342454 ,
        -0.19780108,  0.40988731],
       ..., 
       [ 0.02394596,  1.84274784, -0.38057392, ..., -0.04018703,
        -0.02864329,  1.14297589],
       [-0.48711327, -0.26746489,  0.04716245, ..., -0.21530735,
        -0.26260887,  0.42527288],
       [ 0.46252422,  0.41959075, -1.10269736, ...,  0.38944256,
        -3.70105486,  1.38381423]])

In [ ]:
A= np.ones((2,3))
B= 2*np.ones((2,2))
C= np.empty((2,))
na,na1=A.shape
na

In [7]:
#Approximate cos(x)
x=Chebnodes((10,0,20),2)
y= np.cos(x)
Phi=evalbasex((10,0,20),x)
#Invert to get polynomial coefficients
coeff= np.linalg.inv(Phi) @ y

x1=Chebnodes((100,0,20),0)
y1=np.cos(x1)
Phi1=evalbasex((10,0,20),x1)
y_approx= Phi1 @ coeff
plt.plot(x1,y1)
plt.plot(x1,y_approx)
plt.show()